In [4]:
#pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.5 MB 9.1 MB/s eta 0:00:02
   --------------- ------------------------ 4.5/11.5 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.8/11.5 MB 11.3 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.5 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 11.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

In [22]:
file_path = r"df_food_clean_final.csv"

df_food = pd.read_csv(file_path, encoding="utf-8")  # 'utf-8' 인코딩 사용

print(df_food.head()) 

                  상호                              주소 상권업종대분류명 상권업종중분류명  \
0  아웃백스테이크하우스건대스타시티점       서울특별시 광진구 능동로 92 (자양동) 2층       음식      서양식   
1         엽기떡볶이건대2호점      서울특별시 광진구 동일로22길 111 (화양동)       음식    기타 간이   
2           60계서울군자점        서울특별시 광진구 면목로 1 (군자동) 1층       음식    기타 간이   
3              60계구의  서울특별시 광진구 광나루로56길 34 (구의동) B1층       음식    기타 간이   
4                고술행    서울특별시 광진구 긴고랑로22길 4 (중곡동) 1층       음식       한식   

   상권업종소분류명          경도         위도              도로명주소        지정일자  지정등급  
0       경양식  127.071200  37.538992      서울특별시광진구능동로92  2023-05-23  매우우수  
1  김밥/만두/분식  127.070238  37.540787  서울특별시광진구동일로22길111  2024-08-09  매우우수  
2        치킨  127.074866  37.554528       서울특별시광진구면목로1  2024-04-15  매우우수  
3        치킨  127.096613  37.538135  서울특별시광진구광나루로56길34  2024-05-29  매우우수  
4    백반/한정식  127.084217  37.560845   서울특별시광진구긴고랑로22길4  2024-05-14  매우우수  


In [10]:
#pip install selenium beautifulsoup4 pandas

  Using cached selenium-4.30.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.30.0-py3-none-any.whl (9.4 MB)
Using cached trio-0.29.0-py3-none-any.whl (492 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  # <- 이 부분 추가!
from selenium.common.exceptions import InvalidSessionIdException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [13]:
chrome_driver_path = "chromedriver.exe"  # chromedriver 경로 지정
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service)

In [27]:
def get_naver_map_menus(df_food, driver):
    try:
        if driver.session_id is None:
            print("🚀 새로운 웹드라이버 세션 생성 중...")
            driver = webdriver.Chrome()
    except AttributeError:
        print("🚀 새로운 웹드라이버 세션 생성 중...")
        driver = webdriver.Chrome()

    menu_data = {}

    for index, row in df_food.iterrows():
        store_name = row["상호"]
        if store_name.endswith("점"):
            search_url = f"https://map.naver.com/v5/search/{store_name}"
        else:
            search_url = f"https://map.naver.com/v5/search/{store_name} 광진구"

        try:
            driver.get(search_url)
        except:
            print("⚠️ 웹드라이버 세션 오류 발생. 다시 실행합니다.")
            driver.quit()
            driver = webdriver.Chrome()
            driver.get(search_url)

        time.sleep(5)

        # 🔹 searchIframe 내부로 진입
        try:
            WebDriverWait(driver, 3).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
            )
        except:
            print(f"{store_name}: searchIframe 로딩 실패. 계속 진행.")
            continue

        # # 검색 리스트에서 첫 번째 가게 클릭
        # try:
        #     store_list = WebDriverWait(driver, 3).until(
        #         EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul'))
        #     )
        #     first_store = store_list.find_element(By.TAG_NAME, 'a')
        #     first_store.send_keys(Keys.ENTER)
        # except:
        #     print(f"{store_name}: 가게 클릭 실패. 계속 진행.")
        #     continue

        # 검색 리스트에서 첫 번째 가게 클릭
        try:
            store_list = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul'))
            )
            first_store = store_list.find_element(By.TAG_NAME, 'a')
            print(f"{store_name}: 가게 클릭 완료")
    
        except:
            # 검색 결과가 하나라면 자동으로 상세 페이지로 이동했을 가능성이 있음
            print(f"{store_name}: 검색 결과 하나 → 자동 이동 감지")

        driver.switch_to.default_content()

        # 🔹 entryIframe 내부로 이동
        try:
            WebDriverWait(driver, 3).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
            )
        except:
            print(f"{store_name}: entryIframe 로딩 실패. 계속 진행.")
            continue

        # 🔥 '메뉴' 버튼 클릭
        try:
            menu_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'menu')]"))
            )
            menu_button.click()
            print(f"✅ {store_name}: 메뉴 버튼 클릭 완료")
        except:
            print(f"{store_name}: 메뉴 버튼을 찾지 못했습니다. 계속 진행.")
            continue

        # 🔹 메뉴 탭이 로딩될 때까지 대기
        time.sleep(3)

        # 🔹 메뉴 리스트 크롤링
        menu_data[store_name] = []
        
        try:
            menu_items = WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.E2jtL"))
            )
            for item in menu_items:
                try:
                    menu_name = item.find_element(By.CSS_SELECTOR, "span.lPzHi").text
                    menu_price = item.find_element(By.CSS_SELECTOR, "div.GXS1X em").text
                    menu_data[store_name].append((menu_name, menu_price + "원"))  # 가격 뒤에 '원' 붙이기
                except:
                    continue
        except:
            try: 
                menu_items = WebDriverWait(driver, 3).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.order_list_item"))
                )
                for item in menu_items:
                    try:
                        menu_name = item.find_element(By.CSS_SELECTOR, "div.tit").text
                        menu_price = item.find_element(By.CSS_SELECTOR, "div.price").text
                        menu_data[store_name].append((menu_name, menu_price + "원"))  # 가격 뒤에 '원' 붙이기
                    except:
                        continue
            except: 
                print(f"{store_name}: 메뉴 데이터 없음")

    return menu_data

In [28]:
my_menu = get_naver_map_menus(df_food, driver)

⚠️ 웹드라이버 세션 오류 발생. 다시 실행합니다.
아웃백스테이크하우스건대스타시티점: 가게 클릭 완료
✅ 아웃백스테이크하우스건대스타시티점: 메뉴 버튼 클릭 완료
엽기떡볶이건대2호점: 검색 결과 하나 → 자동 이동 감지
엽기떡볶이건대2호점: entryIframe 로딩 실패. 계속 진행.
60계서울군자점: 가게 클릭 완료
✅ 60계서울군자점: 메뉴 버튼 클릭 완료
60계구의: 검색 결과 하나 → 자동 이동 감지
60계구의: entryIframe 로딩 실패. 계속 진행.
고술행: 검색 결과 하나 → 자동 이동 감지
고술행: entryIframe 로딩 실패. 계속 진행.
교촌치킨: 가게 클릭 완료
교촌치킨: entryIframe 로딩 실패. 계속 진행.
교촌치킨중곡2호: 가게 클릭 완료
교촌치킨중곡2호: entryIframe 로딩 실패. 계속 진행.
군자사랑파리바게트: 가게 클릭 완료
✅ 군자사랑파리바게트: 메뉴 버튼 클릭 완료
굽네치킨중곡4동: 가게 클릭 완료
굽네치킨중곡4동: entryIframe 로딩 실패. 계속 진행.
길동우동건대점: 가게 클릭 완료
✅ 길동우동건대점: 메뉴 버튼 클릭 완료
나폴레옹과자점건대점: 가게 클릭 완료
나폴레옹과자점건대점: entryIframe 로딩 실패. 계속 진행.
더벤티중곡점: 가게 클릭 완료
✅ 더벤티중곡점: 메뉴 버튼 클릭 완료
더벤티구의: 가게 클릭 완료
더벤티구의: entryIframe 로딩 실패. 계속 진행.
던킨군자역점: 가게 클릭 완료
던킨군자역점: entryIframe 로딩 실패. 계속 진행.
비알던킨도너츠건대입구역점: 검색 결과 하나 → 자동 이동 감지
비알던킨도너츠건대입구역점: entryIframe 로딩 실패. 계속 진행.
던킨도너츠: 가게 클릭 완료
던킨도너츠: entryIframe 로딩 실패. 계속 진행.
던킨도너츠테크노마트: 가게 클릭 완료
던킨도너츠테크노마트: entryIframe 로딩 실패. 계속 진행.
던킨: 가게 클릭 완료
던킨: entryIframe 로딩 실패. 계속 진행.
도미노피자군자점: 가게 

In [29]:
for each in my_menu:
    print(my_menu[each])

[('토마호크 7 프리미어 에디션', '335,000원'), ('포터하우스 (100g 당)', '30,000원'), ('블랙라벨 킹 프라운 패밀리세트', '189,000원')]
[]
[('소금버터 소보루번', '2,500원원'), ('카페모카빵', '4,900원원'), ('쫀득치즈스틱(3개입)', '3,200원원'), ('에그 소시지 베이글 샌드위치', '4,900원원'), ('꽈배기도넛', '2,200원원'), ('인생 단팥생크림빵', '3,300원원'), ('인생 소보로생크림빵', '3,300원원'), ('옛날왕슈크림도넛', '2,600원원'), ('달콤부드러운 후레쉬크림샌드빵', '5,800원원'), ('치즈소시지페스츄리', '3,800원원'), ('슈크림빵', '2,000원원'), ('추억의소시지빵', '3,500원원'), ('더부드러워진 아다지오 커피번', '3,700원원'), ('치즈 듬뿍 어니언', '4,500원원'), ('슈크림슈', '1,500원원'), ('메론크림빵', '3,100원원'), ('돌돌말린소시지도넛', '3,500원원'), ('결이 살아있는 모닝토스트', '6,300원원'), ('뉴욕치즈타르트', '2,600원원'), ('블랙올리브치즈치아바타', '3,600원원'), ('밀푀유 슈케이크 (조각)', '6,100원원'), ('15겹 크레이프 케이크 (조각)', '6,100원원'), ('블랙올리브치즈치아바타 샌드위치', '5,100원원'), ('부드러운 시골치즈빵(1/2)', '4,300원원'), ('버터롤 (5개입)', '3,700원원'), ('스위트밀크롤 (8개입)', '3,400원원'), ('사르르 생크림 수플레', '3,900원원'), ('바질리코타 호밀 샌드위치', '6,700원원'), ('치킨에그 통곡물 샌드위치', '7,500원원'), ('골드숫자초 2호', '1,200원원'), ('리본 고깔모자', '1,800원원'), ('골드숫자초 6호', '1,200원원'), ('골드숫자초 9호', '1,200원원'), ('골드숫자

In [36]:
count =0
for store, menus in my_menu.items():
    if len(menus)==0:
        count += 1;
        print(f"{store}: {len(menus)}개")

60계서울군자점: 0개
뚜레쥬르건국대병원점: 0개
한국맥도날드어린이대공원점: 0개
맥도날드어린이대공원점: 0개


In [37]:
count =0
for store, menus in my_menu.items():
    count += 1;
    print(f"{store}: {len(menus)}개")
print(f"{count}개의 결과")

아웃백스테이크하우스건대스타시티점: 3개
60계서울군자점: 0개
군자사랑파리바게트: 249개
길동우동건대점: 32개
더벤티중곡점: 15개
도미노피자군자점: 159개
도제: 15개
두찜건대점: 44개
뚜레쥬르건국대병원점: 0개
래빗홀버거컴퍼니: 6개
맘스터치광장동점: 13개
한국맥도날드어린이대공원점: 0개
맥도날드어린이대공원점: 0개
배스킨라빈스건대스타시티점: 93개
백돈구의점: 55개
백억커피건대로데오점: 1개
버거킹건대입구역점: 122개
버거킹구의점: 122개
베스킨라빈스광장현대점: 112개
빕스어린이대공원점: 29개
빽다방광진이튼1차점: 60개
빽다방자양전통시장점: 10개
슬로우캘리구의역점: 26개
신전떡볶이중곡점: 32개
아빠찜구의역점: 20개
옥루몽어린이대공원점: 43개
유유샤브군자역점: 15개
이디야커피구의자양로점: 91개
이디야커피자양동점: 321개
이디야커피동서울: 200개
해초섬롯데건대스타시티: 4개
청년피자구의직영점: 51개
청년피자중곡직영점: 51개
컴포즈커피건대양꼬치거리점: 51개
컴포즈커피건대스타시티몰점: 1개
크리스피크림건대스타시티점: 12개
투썸플레이스중곡대원외고점: 5개
투썸플레이스구의역점: 1개
팔자좀피자: 27개
푸라닭화양점: 106개
푸라닭광장점: 106개
푸라닭자양점: 106개
플러스82건대점: 1개
하이오커피군자역점: 63개
한결보쌈: 34개
홍대개미롯데스타시티점: 9개
46개의 결과


In [ ]:
#driver.quit()